## Starting a Coroutine

In [2]:
import asyncio

async def coroutine():
    print('in coroutine')
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(coroutine())
finally:
    event_loop.close()

in coroutine


## Returning Values from Coroutines

In [3]:
import asyncio
async def coroutine():
    print('in coroutine')
    return 1234
event_loop = asyncio.get_event_loop()
try:
    result = event_loop.run_until_complete(coroutine())
    print(result)
finally:
    event_loop.close()

in coroutine
1234


## Chaining Coroutines

In [1]:
import asyncio
async def outter():
    result1 = await phase1()
    result2 = await phase2(result1)
    return result1, result2
    
async def phase1():
    return 'hahha'

async def phase2(arg):
    return arg+'recived'

event_loop = asyncio.get_event_loop()
try:
    result = event_loop.run_until_complete(outter())
    print(result)
finally:
    event_loop.close()

('hahha', 'hahharecived')


## Scheduling a Callback “Soon”

In [1]:
import asyncio
import functools
def callback(arg, *, kwargs='default'):
    print(f"callback with {arg} and {kwargs}")
    
async def main(loop):
    print('registering callbacks')
    loop.call_soon(callback, 1)
    wrapped = functools.partial(callback, kwargs='not_default')
    loop.call_soon(wrapped, 2)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

registering callbacks
callback with 1 and default
callback with 2 and not_default


## Scheduling a Callback with a Delay

In [1]:
import asyncio
def callback(n):
    print(f"callback with {n} ")
    
async def main(loop):
    print('registering callbacks')
    loop.call_later(0.2,callback, 1)
    loop.call_later(0.1,callback, 2)
    loop.call_soon(callback, 3)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

registering callbacks
callback with 3 
callback with 2 
callback with 1 


## Scheduling a Callback for a Specific Time

In [1]:
import asyncio
import time
def callback(n, loop):
    print(f"callback with {n} at {loop.time()} ")
    
async def main(loop):
    now = loop.time()
    print('clock time ', time.time())
    print('loop time ', now)
    loop.call_at(0.2+now,callback, 1, loop)
    loop.call_at(0.1+now,callback, 2, loop)
    loop.call_soon(callback, 3, loop)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

clock time  1538278157.53533
loop time  149226.490097989
callback with 3 at 149226.490335996 
callback with 2 at 149226.592940694 
callback with 1 at 149226.693366884 


## Waiting for a Future

In [1]:
import asyncio

def mark_done(future, result):
    print(f"setting future result for {result!r}")
    future.set_result(result)
    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    print('scheduling mark_done')
    event_loop.call_soon(mark_done, all_done, 'the result111')
    result = event_loop.run_until_complete(all_done)
    print('return result',result)
finally:
    event_loop.close()
print('final result ',all_done.result())

scheduling mark_done
setting future result for 'the result111'
return result the result111
final result  the result111


In [1]:
# future with await
import asyncio

def mark_done(future, result):
    print(f"setting future result for {result!r}")
    future.set_result(result)
    
async def main(loop):
    all_done = asyncio.Future()
    print('scheduling mark_done')
    loop.call_soon(mark_done, all_done, 'the result222')
    result =  await all_done
    print('return result',result)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

scheduling mark_done
setting future result for 'the result222'
return result the result222


## Future Callbacks

In [ ]:
import asyncio
import functools

def callback(future, n):
    print(f"{n}: future done: {future.result()}")
    
async def reg_callbacks(all_done):
    all_done.add_done_callback(functools.partial(callback, n=1))
    all_done.add_done_callback(functools.partial(callback, n=3))
    
async def main(all_done):
    await reg_callbacks(all_done)
    all_done.set_result('sdasdsa')
    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    event_loop.run_until_complete(main(all_done))
finally:
    event_loop.close()